# Setup
## Importing Dependencies 

In [1]:
# standard dependencies
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Siamese Neural network for one shot image Recognetion
# import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

## Create Folder Structures

In [3]:
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [5]:
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'data\\positive'

# Collect Positives and Anchors

In [21]:
# extract data set
!tar -xf lfw.tgz

In [24]:
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directory)):
        EX_PATH = os.path.join('lfw', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

## Collect Positives and Anchors classes

In [4]:
# import uuid to generate unique names
import uuid

In [86]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    
    # cut down frame to 250x250
    frame = frame[150:150+250, 230:230+250, :]
    
    # collect anchors
    if cv2.waitKey(1) & 0XFF == ord('a'):
        imagename = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imagename, frame)

    #collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        imagename = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imagename, frame)

    
    cv2.imshow('Image Collection', frame)
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# Load and process images

##  Image directories

In [5]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'\*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH+'\*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH+'\*.jpg').take(300)

In [6]:
dir_test = anchor.as_numpy_iterator()
dir_test.next()

b'data\\anchor\\8679d467-c5e5-11ed-a418-40a3cc81915c.jpg'

##  Preprocessing - Scale and Resize

In [7]:
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img,(100,100)) # 100x100x3
    
    # Scale image to be between 0 and 1
    img = img / 255.0
    
    return img

In [8]:
img = preprocess('data\\anchor\\9cba0f70-c5e5-11ed-a88d-40a3cc81915c.jpg')

In [9]:
img.numpy().min(), img.numpy().max()

(0.0004901961, 0.9411765)

##  Create Labelled DataSet

In [10]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)


In [11]:
sample = data.as_numpy_iterator()
example = sample.next()
example

(b'data\\anchor\\9216b23b-c5e5-11ed-ae3d-40a3cc81915c.jpg',
 b'data\\positive\\b1e58c16-c5e5-11ed-835e-40a3cc81915c.jpg',
 1.0)

In [12]:
# (anchor, positive) => 1,1,1,1,1
# (anchor, negative) => 0,0,0,0,0

## Build Train and Test Partition

In [13]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img),label)

In [14]:
res = preprocess_twin(*example)

In [15]:
res[2]

1.0

In [16]:
# Build DataLoader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=1024)

In [17]:
# Training partition
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [18]:
# Testing partition
test_data = data.skip(round(len(data)*.7))
test_data = data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

# Model engineering

##  Build Embedding Layer

In [19]:
def make_embedding():
    
    inp = Input(shape=(100,100,3), name='input_image')
    
    # first block
    c1 = Conv2D(64,(10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)
    
    # second block
    c2 = Conv2D(128,(7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
    
    # third block
    c3 = Conv2D(128,(4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    
    # final embedding block
    c4 = Conv2D(256,(4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)

    
    return Model(inputs = [inp], outputs = [d1], name = 'embedding')

In [20]:
embedding = make_embedding()

In [21]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_image (InputLayer)     [(None, 100, 100, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 91, 91, 64)        19264     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 46, 46, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 40, 40, 128)       401536    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 128)       262272    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 9, 9, 128)         0 

## Build Distance Layer

In [22]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
        
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

In [23]:
l1 = L1Dist()

##  Make Siamese Model

In [24]:
input_image =  Input(shape=(100,100,3), name='input_img')
validation_image = Input(shape=(100,100,3), name='validation_img')

In [25]:
inp_embedding = embedding(input_image)
val_embedding = embedding(validation_image)

In [26]:
siamese_layer = L1Dist()

In [27]:
distances = siamese_layer(inp_embedding, val_embedding)

In [28]:
classifier = Dense(1, activation='sigmoid')(distances)
classifier

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'dense_1')>

In [29]:
def make_siamese_model():
    
    input_image =  Input(shape=(100,100,3), name='input_img')
    validation_image = Input(shape=(100,100,3), name='validation_img')
    
    # combine siamese distance components
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classification Layer
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs = [input_image, validation_image], outputs = classifier, name="SiameseNetwork")

In [30]:
siamese_model = make_siamese_model()

In [31]:
siamese_model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_img (InputLayer)          [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
validation_img (InputLayer)     [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
embedding (Functional)          (None, 4096)         38960448    input_img[0][0]                  
                                                                 validation_img[0][0]             
__________________________________________________________________________________________________
distance (L1Dist)               (None, 4096)         0           embedding[2][0]     

# Training 

## Setup Loss and Optimizer

In [32]:
binary_cross_loss = tf.losses.BinaryCrossentropy()

In [33]:
opt = tf.keras.optimizers.Adam(1e-4)

## Checkpoints

In [34]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt = opt, siamese_model = siamese_model)

## Train Step Function

In [35]:
test_batch = train_data.as_numpy_iterator()
batch1 = test_batch.next()
X = batch1[:2]


In [36]:
Y = batch1[2]

In [37]:
@tf.function
def train_step(batch):  
    # Record operations
    with tf.GradientTape() as tape:
        # Get anchor and positive/negative image
        x = batch[:2]
        # Get label
        y = batch[2]
        
        # Forward pass
        yhat = siamese_model(x, training=True)
        # Calculate loss
        loss = binary_cross_loss(y, yhat)
    
    grad = tape.gradient(loss, siamese_model.trainable_variables) 
    # Calculate updated weights and apply it to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
    
    return loss

## Trainig loop

In [38]:
def train(data, EPOCHS):
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}\{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        for idx, batch in enumerate(data):
            train_step(batch)
            progbar.update(idx+1)
            
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)
        

## Train the model

In [41]:
EPOCHS = 50

In [42]:
train(train_data, EPOCHS)


 Epoch 1\50
27/27 [==============================] - 1169s 43s/step

 Epoch 2\50
27/27 [==============================] - 1143s 42s/step

 Epoch 3\50
27/27 [==============================] - 1147s 42s/step

 Epoch 4\50
27/27 [==============================] - 945s 35s/step

 Epoch 5\50
27/27 [==============================] - 498s 18s/step

 Epoch 6\50
27/27 [==============================] - 498s 18s/step

 Epoch 7\50
27/27 [==============================] - 500s 18s/step

 Epoch 8\50
27/27 [==============================] - 498s 18s/step

 Epoch 9\50
27/27 [==============================] - 489s 18s/step

 Epoch 10\50
27/27 [==============================] - 476s 18s/step

 Epoch 11\50
27/27 [==============================] - 477s 18s/step

 Epoch 12\50
27/27 [==============================] - 477s 18s/step

 Epoch 13\50
27/27 [==============================] - 476s 18s/step

 Epoch 14\50
27/27 [==============================] - 477s 18s/step

 Epoch 15\50
27/27 [===================

# Evaluate Model

In [39]:
# Import metric calculations
from tensorflow.keras.metrics import Precision, Recall

# Get a batch of test data
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

## Prediction

In [40]:
y_hat = siamese_model.predict([test_input, test_val])
y_hat

array([[0.50055707],
       [0.5002263 ],
       [0.5019772 ],
       [0.50075376],
       [0.50042593],
       [0.50021875],
       [0.49969372],
       [0.5004967 ],
       [0.49961907],
       [0.49958998],
       [0.50070053],
       [0.5015157 ],
       [0.50006783],
       [0.50017244],
       [0.50039524],
       [0.50004977]], dtype=float32)

In [41]:
[1 if prediction > 0.5 else 0 for prediction in y_hat ]

[1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1]

In [42]:
y_true

array([0., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.],
      dtype=float32)

## Calculate Metrics

In [43]:
m = Recall() # or Precision()

m.update_state(y_true, y_hat)
m.result().numpy()

0.72727275

## Vizualisation Results

In [44]:
#plt.figure(figsize=(10,8))
#plt.subplot(1,2,1)
#plt.imshow(test_input[0])
#plt.subplot(1,2,2)
#plt.imshow(test_val[8])
#plt.show()

# Save Model

In [26]:
siamese_model.save('siameseModel.h5')

NameError: name 'siamese_model' is not defined

In [45]:
model = tf.keras.models.load_model('siameseModel.h5', custom_objects= {'L1Dist':L1Dist, 'BinaryCrosstropy':tf.losses.BinaryCrossentropy})

In [46]:
model.predict([test_input, test_val])

array([[1.1569635e-09],
       [9.9996823e-01],
       [2.8518587e-09],
       [3.0154096e-10],
       [6.8840201e-08],
       [9.9992388e-01],
       [1.0000000e+00],
       [3.7865253e-09],
       [9.9996430e-01],
       [1.0000000e+00],
       [9.9998623e-01],
       [9.9973822e-01],
       [1.0000000e+00],
       [9.9942636e-01],
       [9.9980456e-01],
       [1.0000000e+00]], dtype=float32)

In [47]:
model.summary()

Model: "SiameseNetwork"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_img (InputLayer)          [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
validation_img (InputLayer)     [(None, 100, 100, 3) 0                                            
__________________________________________________________________________________________________
embedding (Functional)          (None, 4096)         38960448    input_img[0][0]                  
                                                                 validation_img[0][0]             
__________________________________________________________________________________________________
l1_dist_3 (L1Dist)              (None, 4096)         0           embedding[0][0]     

#  Real Time Test

## Verification function

In [48]:
def verify(model, detection_threshold, verification_threshold):
    results = []
    for image in os.listdir(os.path.join('application_data','verification_images')):
        input_img = preprocess(os.path.join('application_data', 'input_image', 'input_image.jpg'))
        validation_img = preprocess(os.path.join('application_data', 'verification_images', image))
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result) 
    detection = np.sum(np.array(results) > detection_threshold)
    verification = detection / len(os.listdir(os.path.join('application_data', 'verification_images')))  
    verified = verification > verification_threshold
    
    return results, verified

## Real Time Detection

In [61]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[150:150+250, 230:230+250, :]

    cv2.imshow('Verification', frame)
    
    if cv2.waitKey(10) & 0XFF == ord('v'):
        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        results, verified = verify(model, 0.7, 0.4)
        print(verified)
        
    if cv2.waitKey(10) & 0XFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

False
True
True


In [52]:
results

[array([[1.9428524e-07]], dtype=float32),
 array([[8.479886e-06]], dtype=float32),
 array([[5.58134e-06]], dtype=float32),
 array([[1.246799e-05]], dtype=float32),
 array([[2.7154689e-05]], dtype=float32),
 array([[0.9581151]], dtype=float32),
 array([[0.9581151]], dtype=float32),
 array([[0.9994899]], dtype=float32),
 array([[0.99867797]], dtype=float32),
 array([[0.99867797]], dtype=float32),
 array([[0.999027]], dtype=float32),
 array([[0.9993745]], dtype=float32),
 array([[0.99975634]], dtype=float32),
 array([[0.9992592]], dtype=float32),
 array([[0.999177]], dtype=float32),
 array([[0.00024498]], dtype=float32),
 array([[0.00068316]], dtype=float32),
 array([[0.00056511]], dtype=float32),
 array([[0.99645376]], dtype=float32),
 array([[0.99880236]], dtype=float32),
 array([[1.2279579e-06]], dtype=float32),
 array([[2.0932418e-06]], dtype=float32),
 array([[2.0932418e-06]], dtype=float32),
 array([[3.3446402e-06]], dtype=float32),
 array([[3.5176906e-06]], dtype=float32),
 array([